<a href="https://colab.research.google.com/github/Sh0g0-1758/Zenith-zealots/blob/main/modified_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install qiskit
!pip install qiskit-aer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 12.0 MB/s eta 0:00:00


In [8]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram
import numpy as np

# Import the necessary functions for visualizing quantum states
from qiskit.visualization import plot_bloch_multivector

# Import the Statevector class from Qiskit's quantum information module
from qiskit.quantum_info import Statevector
import numpy as np


# def matrix_gen(qubit_location, valid_qubits):
#     # initialise the matrix with all -1
#     matrix = np.full((4, 4), -1)

#
#     # set the qubit locations
#     for qubit in qubit_location:
#         if valid_qubits[qubit]:
#             matrix[qubit_location[qubit][0]][qubit_location[qubit][1]] = qubit
#
#     return matrix

def print_matrix(matrix):
    for i in range(4):
        for j in range(4):
            print(matrix[i][j], end=" ")
        print()
    print()


def left_shift(matrix):
    n = len(matrix)
    # array of length n with 0
    last_loc = np.zeros(n)

    for i in range(n):
        for j in range(n):
            if matrix[i][j] != -1:
                val = matrix[i][j]
                matrix[i][j] = -1
                # shift it to last-loc in that row
                matrix[i][int(last_loc[i])] = val
                last_loc[i] += 1

    return matrix


def right_shift(matrix):
    n = len(matrix)
    # array of length n with (n-1)
    last_loc = np.full(n, n - 1)

    for i in range(n):
        for j in range(n - 1, -1, -1):
            if matrix[i][j] != -1:
                val = matrix[i][j]
                matrix[i][j] = -1
                # shift it to last-loc in that row
                matrix[i][int(last_loc[i])] = val
                last_loc[i] -= 1
    return matrix


def up_shift(matrix):
    n = len(matrix)
    # array of length n with 0
    last_loc = np.zeros(n)

    for i in range(n):
        for j in range(n):
            if matrix[j][i] != -1:
                val = matrix[j][i]
                matrix[j][i] = -1
                # shift it to last-loc in that row
                matrix[int(last_loc[i])][i] = val
                last_loc[i] += 1

    return matrix


def down_shift(matrix):
    n = len(matrix)
    # array of length n with n-1
    last_loc = np.full(n, n - 1)

    for i in range(n):
        for j in range(n - 1, -1, -1):
            print("row: ", j, "col: ", i)
            if matrix[j][i] != -1:
                val = matrix[j][i]
                matrix[j][i] = -1
                # shift it to last-loc in that row
                matrix[last_loc[i]][i] = val
                last_loc[i] -= 1

    return matrix


def shift(matrix, direction):
    if direction == 0:
        return left_shift(matrix)
    elif direction == 1:
        return right_shift(matrix)
    elif direction == 2:
        return up_shift(matrix)
    elif direction == 3:
        return down_shift(matrix)
    else:
        print("Invalid direction")
        return matrix


# def operate_gate(qc_map,qc_loc,qc_matrix, direction, gates):

def merge_left_wall(qc_loc, qc_matrix, state_vector_dict):
    j = 0
    k = 1
    for i in range(len(qc_matrix)):
        if qc_matrix[i][j] == -1 or qc_matrix[i][k] == -1:
            continue
        s1 = state_vector_dict[qc_matrix[i][j]]
        s2 = state_vector_dict[qc_matrix[i][k]]
        if s1 == s2:
            # we will non-wall qubit
            del qc_map[qc_matrix[i][k]]
            qc_loc[qc_matrix[i][k]] = None
            qc_matrix[i][k] = -1

def merge_right_wall(qc_loc, qc_matrix, state_vector_dict):
    j = len(qc_matrix) - 1
    k = len(qc_matrix) - 2
    for i in range(len(qc_matrix)):
        if qc_matrix[i][j] == -1 or qc_matrix[i][k] == -1:
            continue
        s1 = state_vector_dict[qc_matrix[i][j]]
        s2 = state_vector_dict[qc_matrix[i][k]]
        if s1 == s2:
            # we will non-wall qubit
            del qc_map[qc_matrix[i][k]]
            qc_loc[qc_matrix[i][k]] = None
            qc_matrix[i][k] = -1

def merge_up_wall(qc_loc, qc_matrix, state_vector_dict):
    j = 0
    k = 1
    for i in range(len(qc_matrix)):
        if qc_matrix[i][j] == -1 or qc_matrix[i][k] == -1:
            continue
        s1 = state_vector_dict[qc_matrix[j][i]]
        s2 = state_vector_dict[qc_matrix[k][i]]
        if s1 == s2:
            # we will non-wall qubit
            del qc_map[qc_matrix[k][i]]
            qc_loc[qc_matrix[k][i]] = None
            qc_matrix[k][i] = -1

def merge_down_wall(qc_loc, qc_matrix, state_vector_dict):
    j = len(qc_matrix) - 1
    k = len(qc_matrix) - 2
    for i in range(len(qc_matrix)):
        if qc_matrix[i][j] == -1 or qc_matrix[i][k] == -1:
            continue
        s1 = state_vector_dict[qc_matrix[j][i]]
        s2 = state_vector_dict[qc_matrix[k][i]]
        if s1 == s2:
            # we will non-wall qubit
            del qc_map[qc_matrix[k][i]]
            qc_loc[qc_matrix[k][i]] = None
            qc_matrix[k][i] = -1


def merge(qc_map, qc_loc, qc_matrix, curr_dir, state_vector_dict):
    # first we update the state vector
    for i in range(len(qc_map)):
        state_vector_dict[i] = Statevector(qc_map[i])

    # now depending on the direction we will iterate the wall and compare their state vectors and if same we will
    # destroy the non-wall qubit
    if curr_dir == 0:
        merge_left_wall(qc_loc, qc_matrix, state_vector_dict)
    elif curr_dir == 1:
        merge_right_wall(qc_loc, qc_matrix, state_vector_dict)
    elif curr_dir == 2:
        merge_up_wall(qc_loc, qc_matrix, state_vector_dict)
    elif curr_dir == 3:
        merge_down_wall(qc_loc, qc_matrix, state_vector_dict)
    else:
        print("Invalid direction")

def matrix_gen(qubit_location):
    # initialise the matrix with all -1
    matrix = np.full((4, 4), -1)

    # set the qubit locations
    for qubit in qubit_location:
        matrix[qubit_location[qubit][0]][qubit_location[qubit][1]] = qubit

    return matrix

def operate_gate(qc_map, qc_i,dir,gate):
    gate = gate[dir]
    if gate == 'H':
        qc_map[qc_i].h(0)
    elif gate == 'X':
        qc_map[qc_i].x(0)
    elif gate == 'I':
        qc_map[qc_i].i(0)
    elif gate == 'Z':
        qc_map[qc_i].z(0)
    else:
        print("Invalid gate")


def gate_op(curr_dir, qc_map, qc_location, qc_matrix, wall_gate):

    # now we will operate the gates
    for i in range(len(qc_matrix)):
        if curr_dir == 0:
            # left
            if qc_matrix[i][0] != -1:
                operate_gate(qc_map, qc_matrix[i][0], curr_dir, wall_gate)
        elif curr_dir == 1:
            # right
            if qc_matrix[i][len(qc_matrix) - 1] != -1:
                operate_gate(qc_map, qc_matrix[i][len(qc_matrix) - 1], curr_dir, wall_gate)
        elif curr_dir == 2:
            # up
            if qc_matrix[0][i] != -1:
                operate_gate(qc_map, qc_matrix[0][i], curr_dir, wall_gate)
        elif curr_dir == 3:
            # down
            if qc_matrix[len(qc_matrix) - 1][i] != -1:
                operate_gate(qc_map, qc_matrix[len(qc_matrix) - 1][i], curr_dir, wall_gate)
        else:
            print("Invalid direction")

    qc_matrix = matrix_gen(qc_location)

In [10]:
last_direction = -1
curr_direction = 0
game_over = False

start_game = curr_direction != last_direction
start_game = start_game & game_over


def process(qc_map, qc_loc, qc_matrix, dir, gates, state_vector):
    # first we will shift the matrix
    shift(qc_matrix, dir)

    # now we will operate the gates
    gate_op(dir, qc_map, qc_loc, qc_matrix, gates)

    # now we will merge the qubits
    merge(qc_map, qc_loc, qc_matrix, dir, state_vector)

    shift(qc_matrix, dir)


# Designing Level 1

wall_gate = {
    0: 'H',
    1: 'X',
    2: 'I',
    3: 'Z'
}

qc1 = QuantumCircuit(1)
qc2 = QuantumCircuit(1)

qc1.h(0)  # Hadamard Gate on qubit |+>
qc1.z(0)  # Setting the initial state to |->
qc2.x(0)  # Setting the initial state to |1>

# create a map of quantum circuits
qc_map = {
    0: qc1,
    1: qc2
}

# now set location of these qubits

qc_location = {
    0: [3, 1],
    1: [3, 3]
}

# valid_qc = [True, True]

# now generate the matrix
location_matrix = matrix_gen(qc_location)

# let's see the matrix
print(location_matrix)

# the shift operation is ready

# creating a state-vector dictionary
state_vector_dict = {}
state_vector_dict[0] = Statevector(qc_map[0])
state_vector_dict[1] = Statevector(qc_map[1])

print(state_vector_dict[0])
print(state_vector_dict[1])

process(qc_map, qc_location, location_matrix, 0, wall_gate, state_vector_dict)

# wall_gate = {
#     0: 'H',
#     1: 'X',
#     2: 'I',
#     3: 'Z'
# }

# qc1.h(0)  # Hadamard Gate on qubit |+>
# qc2.x(0)  # Setting the initial state to |1>

# qc1_test = qc_map[0]
# qc1_test.draw(output='mpl')


print(state_vector_dict[0])

print(location_matrix)


[[-1 -1 -1 -1]
 [-1 -1 -1 -1]
 [-1 -1 -1 -1]
 [-1  0 -1  1]]
Statevector([ 0.70710678+0.j, -0.70710678+0.j],
            dims=(2,))
Statevector([0.+0.j, 1.+0.j],
            dims=(2,))
Statevector([0.+0.j, 1.+0.j],
            dims=(2,))
[[-1 -1 -1 -1]
 [-1 -1 -1 -1]
 [-1 -1 -1 -1]
 [ 0 -1 -1 -1]]


In [6]:
def gate_op(curr_dir, qc_map,qc_location, location_matrix, wall_gate) :
  if curr_dir==0 :
    for i in qc_location:
      if qc_map[i]==None :
        continue
      if qc_location[i][1]==0:
        if wall_gate[0]=='X':
          qc_map[i].x(0)
        if wall_gate[0]=='H':
          qc_map[i].h(0)
        if wall_gate[0]=='Z':
          qc_map[i].z(0)
  if curr_dir==1 :
    for i in qc_location:
      if qc_location[i][1]==3:
        if wall_gate[1]=='X':
          qc_map[i].x(0)
        if wall_gate[1]=='H':
          qc_map[i].h(0)
        if wall_gate[1]=='Z':
          qc_map[i].z(0)
  if curr_dir==2 :
    for i in qc_location:
      if qc_location[i][0]==0:
        if wall_gate[2]=='X':
          qc_map[i].x(0)
        if wall_gate[2]=='H':
          qc_map[i].h(0)
        if wall_gate[2]=='Z':
          qc_map[i].z(0)

  if curr_dir==3 :
    for i in qc_location:
      if qc_location[i][0]==3:
        if wall_gate[3]=='X':
          qc_map[i].x(0)
        if wall_gate[3]=='H':
          qc_map[i].h(0)
        if wall_gate[3]=='Z':
          qc_map[i].z(0)

